In [36]:
import subprocess
import pandas as pd

def monitor(subscription_id):
    bashCmd = 'az vm list --subscription b8871872-a5e3-473f-b9b9-f4baaab6a9a0 -d -o tsv --query "[?powerState=='VM running'].{resource:resourceGroup, name:name'

    process = subprocess.Popen(bashCmd.split(), stdout=subprocess.PIPE)
    
    output, error = process.communicate()
    return output.decode("utf-8") 

data = monitor("b8871872-a5e3-473f-b9b9-f4baaab6a9a0")

SyntaxError: invalid syntax (<ipython-input-36-c9eda1e75d3c>, line 5)

''

In [29]:
from io import StringIO
import pandas as pd

TESTDATA = StringIO(data)
df = pd.read_csv(TESTDATA, sep="\t")
df.head()

,None,None.1,None.2,Unnamed: 3,None.3,None.4,None.5,demofede.centralus.cloudapp.azure.com,Unnamed: 8,None.6,...,40.69.149.250,FEDE-PLAYGROUND,2,None.14,Unnamed: 30,None.15,Microsoft.Compute/virtualMachines,None.16,145ddebc-5375-4be5-baa3-a876bffe9738,None.17
0,None,None,None,None,None,None,None,placelinking.centralus.cloudapp.azure.com,NaN,None,...,40.122.49.68,FEDE-PLAYGROUND,3,None,NaN,NaN,Microsoft.Compute/virtualMachines,None,1f035c3a-6eb9-46e7-aece-17837eaaabf6,None
1,None,None,None,NaN,None,None,None,NaN,NaN,None,...,NaN,DAVE-PLAYGROUND,2,None,NaN,NaN,Microsoft.Compute/virtualMachines,None,cca9ddcf-d242-4f71-b0af-14405ecef540,None
2,None,None,None,NaN,None,None,None,NaN,NaN,None,...,40.127.190.255,DVS-PLAYGROUND,1,None,NaN,None,Microsoft.Compute/virtualMachines,None,babeeee9-ff55-4010-b354-4b5f3f4a26e4,None
3,None,None,None,NaN,None,None,None,NaN,NaN,None,...,137.135.140.164,JISC-INGEST,1,None,NaN,None,Microsoft.Compute/virtualMachines,None,e6f42e00-b43c-4c09-9ef6-a6cf8ec2ec11,None
4,None,None,None,NaN,None,None,None,NaN,NaN,None,...,NaN,MAPS-INFERENCE,2,None,NaN,None,Microsoft.Compute/virtualMachines,None,299f4022-34b3-47c8-b4b2-5e0db21fa62e,None
